In [1]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridBox, Layout
import serial

# Try to open serial connection safely
try:
    ser = serial.Serial('/dev/ttyUSB1', 9600, timeout=1)
except serial.SerialException as e:
    print(f"Error opening serial port: {e}")
    ser = None  # Prevents crashes if the serial port is unavailable

def on_button_clicked(b):
    """Handles button clicks to toggle LED states and send UART commands."""
    
    # Toggle button color (light green for center, light red for others)
    if b.style.button_color == 'lightgray':
        if b.description == "LED 3":
            b.style.button_color = 'lightgreen'
        else:
            b.style.button_color = 'lightcoral'
    else:
        b.style.button_color = 'lightgray'

    # Extract LED number (handles potential extra spaces)
    led_number = b.description.replace('LED', '').strip()

    # Determine desired state
    state = "ON" if b.style.button_color != 'lightgray' else "OFF"

    # Build command string
    command = f"LED{led_number} {state}\n"
    #print("Sending command:", command)  # Debug print; remove in production

    # Send command over UART
    if ser:  # Check if serial port was opened successfully
        ser.write(command.encode('utf-8'))
    else:
        print("Serial port not open. Command not sent.")

# Create LED buttons
buttons = {
    'up': widgets.Button(description='LED 1', layout=Layout(width='60px', height='50px')),
    'left': widgets.Button(description='LED 2', layout=Layout(width='60px', height='50px')),
    'center': widgets.Button(description='LED 3', layout=Layout(width='60px', height='50px')),
    'right': widgets.Button(description='LED 4', layout=Layout(width='60px', height='50px')),
    'down': widgets.Button(description='LED 5', layout=Layout(width='60px', height='50px'))
}

# Initialize all buttons and attach click handler
for button in buttons.values():
    button.style.button_color = 'lightgray'
    button.on_click(on_button_clicked)

# Define grid layout for buttons
grid = GridBox(
    children=[
        widgets.Label(), buttons['up'], widgets.Label(),          # Row 1
        buttons['left'], buttons['center'], buttons['right'],      # Row 2
        widgets.Label(), buttons['down'], widgets.Label()          # Row 3
    ],
    layout=Layout(
        display='grid',
        grid_template_columns="auto auto auto",
        grid_template_rows="auto auto auto",
        justify_content="center",
        align_content="center",
        gap="0px"
    )
)

# Display the GUI
display(grid)

# Ensure serial port closes when script exits
import atexit
if ser:
    atexit.register(ser.close)


GridBox(children=(Label(value=''), Button(description='LED 1', layout=Layout(height='50px', width='60px'), sty…